<a href="https://colab.research.google.com/github/dropcreations/7zip-in-Google-Colab/blob/main/7zip-in-Google-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# __7-Zip__

* Run below cell to **Install 7-Zip** to the runtime

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/7zip-in-Google-Colab/main/7zip-Logo.png' height="50" alt="7zip-logo"/></center>
#@markdown <center><h3><b>Install 7-Zip</b></h3></center><br>
from IPython.display import clear_output
!sudo apt update
!sudo apt install p7zip-full p7zip-rar unrar rar
clear_output()
print("Successfully Installed.")

### __Compress Files and Folders__

* Create **zip, tar, 7z, gz, bz2, xz, wim** files.
* If you want you can add **password** or **split** the archive.
* If you want to save archive in **another** location **uncheck** `saveToSourceLocation`.

In [ ]:
import os

sourcePath = "" #@param {type:"string"}
compressType = "zip" #@param ["zip", "7z", "tar", "gzip", "bzip2", "xz", "wim"]
Password = "" #@param {type:"string"}
Split = "no" #@param ["no", "10m", "100m", "500m", "1g", "2g"] {allow-input: true}
compressLevel = 9 #@param {type:"slider", min:0, max:9, step:1}
saveToSourceLocation = True #@param {type:"boolean"}

commandLine = "-t" + compressType + " -mx=" + str(compressLevel)

if len(Password) > 0:
    commandLine = commandLine + " -p" + '"' + Password + '"'

if Split is not "no":
    commandLine = commandLine + " -v" + '"' + Split + '"'

if os.path.isfile(sourcePath) is True:
    sourceName = os.path.splitext(os.path.basename(os.path.abspath(sourcePath)))[0]
    sourceFolder = os.path.dirname(os.path.abspath(sourcePath))
else:
    sourceName = os.path.split(os.path.abspath(sourcePath))[1]
    sourceFolder = os.path.split(os.path.abspath(sourcePath))[0]

if saveToSourceLocation == True:
    compressPath = os.path.join(sourceFolder, "compressed")
    commandLine = commandLine + ' "' + compressPath + '"'
else:
    outputPath = input("outputPath: ")
    if outputPath.endswith('.zip') or outputPath.endswith('.7z') or outputPath.endswith('.tar') or outputPath.endswith('.gz') or outputPath.endswith('.bz2') or outputPath.endswith('.xz') or outputPath.endswith('.wim'):
        sourceName = os.path.splitext(os.path.basename(os.path.abspath(outputPath)))[0]
        sourceFolder = os.path.dirname(os.path.abspath(outputPath))
    else:
        if os.path.exists(outputPath) is False:
            os.makedirs(outputPath)
        sourceFolder = outputPath
    compressPath = os.path.join(sourceFolder, "compressed")
    commandLine = commandLine + ' "' + compressPath + '"'

if compressType == "gzip":
    formatFile = "gz"
elif compressType == "bzip2":
    formatFile = "bz2"
else:
    formatFile = compressType

!7z a {commandLine} "{sourcePath}"
saveFile = os.path.join(sourceFolder, sourceName)
compressPath = compressPath + '.' + formatFile
saveFile = saveFile + '.' + formatFile
os.rename(compressPath, saveFile)

### __Uncompress Files__

* To **list content** of the file, use `viewFile`. ***Uncheck this after viewing the content***.
* Can also extract **splited** archives.
* `saveToSourceLocation`: Extracts files to source location.
* If you want to extract files from archive **without using directory names**, uncheck `directoryNames`.
> * NOTE : ***Don't uncheck*** `directoryNames` at normal use.
>
>  ![directory_names](https://raw.githubusercontent.com/dropcreations/Essential-Google-Colab-Notebook/main/cell_logos/directory_names.png) ![without_directory_names](https://raw.githubusercontent.com/dropcreations/Essential-Google-Colab-Notebook/main/cell_logos/without_directory_names.png)
>
> * Without `directoryNames`, extracts files as **2nd figure**.

In [ ]:
import os

sourceFile = "" #@param {type:"string"}
viewFile = False #@param {type:"boolean"}
directoryNames = True #@param {type:"boolean"}
saveToSourceLocation = True #@param {type:"boolean"}

sourceName = os.path.splitext(os.path.basename(os.path.abspath(sourceFile)))[0]
sourceFolder = os.path.dirname(os.path.abspath(sourceFile))

if viewFile is True:
    commandLine = 'l "' + sourceFile + '"'
else:
    makeFolder = input("Do you want to create a folder to extract? [y/n] ")
    if saveToSourceLocation is True:
        if makeFolder.lower() == "y":
            extractFolder = os.path.join(sourceFolder, sourceName)
            if os.path.exists(extractFolder) is False:
                os.makedirs(extractFolder, exist_ok=True)
        elif makeFolder.lower() == "n":
            extractFolder = sourceFolder
    else:
        outputPath = input("outputFolder: ")
        if makeFolder.lower() == "y":
            extractFolder = os.path.join(outputPath, sourceName)
            if os.path.exists(extractFolder) is False:
                os.makedirs(extractFolder, exist_ok=True)
        elif makeFolder.lower() == "n":
            extractFolder = outputPath
    if directoryNames is True:
        commandLine = 'x "' + sourceFile + '" -o"' + extractFolder + '"'
    else:
        commandLine = 'e "' + sourceFile + '" -o"' + extractFolder + '"'

!7z {commandLine}